In [ ]:
import os
import pandas as pd
import numpy as np
#Pull NPS API data
import requests
import json
#Database connectivity
from sqlalchemy import create_engine

#Import api key and values for database connectivity
from config import (nps_api_key, user, password, host, port, database)


In [ ]:
path_xlsx = os.getcwd()
files_xlsx = os.listdir(path_xlsx+"/data")

filepath_xlsx = [];
for f in files_xlsx:
    if f[-4:] == 'xlsx':
        filepath_xlsx.append(f'data/{f}');
                    
filepath_xlsx

In [ ]:
nps_df = pd.DataFrame()
nps_df = nps_df.iloc[0:0]

In [ ]:
for f in filepath_xlsx:
    nps_data = pd.read_excel(f, 'Current Year Monthly and Annual', header=11, skipfooter=6)
    if len(nps_df) == 0:
        nps_df = nps_df.append(nps_data)
    else:
        nps_df = nps_df.merge(nps_data, how="inner", on="Park")

In [ ]:
clean_nps_df = nps_df[['Park'
                       ,'MAR 2021', 'MAR 2020'
                       ,'FEB 2021', 'FEB 2020'
                       ,'JAN 2021', 'JAN 2020'
                       ,'DEC 2020', 'DEC 2019'
                       ,'NOV 2020', 'NOV 2019'
                       ,'OCT 2020', 'OCT 2019'
                       ,'SEP 2020', 'SEP 2019'
                       ,'AUG 2020', 'AUG 2019'
                       ,'JUL 2020', 'JUL 2019'
                       ,'JUN 2020', 'JUN 2019'
                       ,'MAY 2020', 'MAY 2019'
                       ,'APR 2020', 'APR 2021']]

np_designations = clean_nps_df['Park'].str[-2:]
    
clean_nps_df.insert(1, "Designation", np_designations, True)

clean_nps_df.head()

In [ ]:
index_to_drop = clean_nps_df[ clean_nps_df["Designation"] != "NP"].index

np_df = clean_nps_df.drop(index_to_drop)

sorted_np_df = np_df.sort_values(["APR 2021"], ascending=False)

sorted_np_df.reset_index(inplace=True)

sorted_np_df["visit_rank"] = np.arange(len(sorted_np_df))+1

clean_np_df = sorted_np_df.drop(["index", "Designation"], axis=1)

clean_np_df.head()

In [ ]:
#Rename columns
clean_np_df.rename(columns={'Park': 'name'
                            ,'MAR 2021' : 'mar2021'
                            , 'MAR 2020': 'mar2020'
                            ,'FEB 2021': 'feb2021'
                            , 'FEB 2020': 'feb2020'
                            ,'JAN 2021': 'jan2021'
                            , 'JAN 2020': 'jan2020'
                            ,'DEC 2020': 'dec2020'
                            , 'DEC 2019': 'dec2019'
                            ,'NOV 2020': 'nov2020'
                            , 'NOV 2019': 'nov2019'
                            ,'OCT 2020': 'oct2020'
                            , 'OCT 2019': 'oct2019'
                            ,'SEP 2020': 'sep2020'
                            , 'SEP 2019': 'sep2019'
                            ,'AUG 2020': 'aug2020'
                            , 'AUG 2019': 'aug2019'
                            ,'JUL 2020': 'jul2020'
                            , 'JUL 2019': 'jul2019'
                            ,'JUN 2020': 'jun2020'
                            , 'JUN 2019': 'jun2019'
                            ,'MAY 2020': 'may2020'
                            , 'MAY 2019': 'may2019'
                            ,'APR 2020': 'apr2020'
                            , 'APR 2021': 'apr2021'                           
                           }, inplace=True)

clean_np_df.head()

In [ ]:
nps_url = f"https://developer.nps.gov/api/v1/parks?limit=500&api_key={nps_api_key}"

nps_url

In [ ]:
response = requests.get(nps_url)
json_response = response.json()

parks = json_response["data"]

In [ ]:
names = []
lats = []
longs = []
urls = []
descriptions = []


for park in parks:
    if (park["designation"] == "National Park" or park["designation"] == "National Parks" or park["designation"] == "National and State Parks"):
        park_name = park["name"]
        names.append(f'{park_name} NP')
        lats.append(park["latitude"])
        longs.append(park["longitude"])
        urls.append(park["url"])
        descriptions.append(park["description"])
    
    

In [ ]:
# create dataframe
np_details_df = pd.DataFrame({
    "name": names,
    "description": descriptions,
    "latitude": lats,
    "longitude": longs,
    "url": urls
})

np_details_df.head()

In [ ]:
final_nps_df = np_details_df.merge(clean_np_df, how="inner", on="name")

final_nps_df.columns

In [ ]:
#Load Database
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
conn = engine.connect()

In [ ]:
engine.table_names()

In [ ]:
engine.execute("delete from parks")

In [ ]:
#Load parks table
final_nps_df.to_sql(name="parks", con=engine, if_exists='append', index=False)

In [ ]:
#Confirm data was loaded into parks table
pd.read_sql_query('select * from parks', con=engine)